In [64]:
import plotly.graph_objects as go
import numpy as np
import os

# モデルのパスや埋め込み範囲などのパラメータの設定

# 22311	mammal.n.01 929	bird.n.01 25214	reptile.n.01　5671	plant.n.01　 12623	fish.n.01
# 26266	lion.n.01 27	dog.n.01 22311	mammal.n.01 26878	cat.n.01 28865	homo_sapiens.n.01 34022	cheetah.n.01 5484	big_cat.n.01 5483	leopard.n.02 28505	wolf.n.01 16709	fox.n.01

model_path = '/home/lab/eight/KGE-HAKE/models/HAKE_wn18rr_0'
embedding_range = 0.01
head_id = 28505 #22311	mammal.n.01 16430	food.n.01 39316	poodle.n.01
tail_id = 16709	#27	dog.n.01 40630	pizza.n.01
num_bins = 250  # ヒストグラムのビンの数
model = 'HAKE'
# エンティティ名の辞書読み込み
entities_dict_file = '/home/lab/eight/KGE-HAKE/data/wn18rr_text/entities.dict'
entities_dict = {}
with open(entities_dict_file, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        entities_dict[int(parts[0])] = parts[1]

In [65]:
# エンティティの埋め込みをロード
entity_embedding = np.load(os.path.join(model_path, 'entity_embedding.npy'))

# HeadとTailの座標計算
head = entity_embedding[head_id]
tail = entity_embedding[tail_id]

phase_head, mod_head = np.split(head, 2)
phase_tail, mod_tail = np.split(tail, 2)

mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

fig = go.Figure()

# Head Entityのプロット
fig.add_trace(go.Scatter(x=x_head, y=y_head, mode='markers', name=entities_dict[head_id], text=[f'Head ID: {head_id}'] * len(x_head), opacity=0.8))
# Tail Entityのプロット
fig.add_trace(go.Scatter(x=x_tail, y=y_tail, mode='markers', name=entities_dict[tail_id], text=[f'Tail ID: {tail_id}'] * len(x_tail), opacity=0.8))
# HAKEテキストの追加
fig.add_annotation(
    text=model,  # 表示するテキスト
    xref="paper",  # x座標の参照元 (paperは図の左端、dataはデータ座標)
    yref="paper",  # y座標の参照元 (paperは図の下端、dataはデータ座標)
    x=0.5,  # x座標 (0.5は図の中央)
    y=1.1,  # y座標 (1.05は図の上端から5%上)
    showarrow=False,  # 矢印を非表示にする
    font=dict(size=20),  # テキストのフォントサイズ
)
# レイアウトの設定
fig.update_layout(
    title="",
    yaxis_title=" ",
    showlegend=True,
    width=550,  # 幅
    height=500,  # 高さ
    xaxis=dict(scaleanchor="y", scaleratio=1),  # x軸のアスペクト比を1:1に設定
    yaxis=dict(scaleanchor="x", scaleratio=1),  # y軸のアスペクト比を1:1に設定
)

# プロットの表示
fig.show()


In [66]:

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

# ログ変換と極座標への変換
mod_head = np.log(np.abs(mod_head)) * np.sign(mod_head)
mod_tail = np.log(np.abs(mod_tail)) * np.sign(mod_tail)

phase_head = phase_head / embedding_range * np.pi
phase_tail = phase_tail / embedding_range * np.pi

# 極座標から直交座標への変換
x_head, y_head = mod_head * np.cos(phase_head), mod_head * np.sin(phase_head)
x_tail, y_tail = mod_tail * np.cos(phase_tail), mod_tail * np.sin(phase_tail)

# 距離の計算
distance_head = np.sqrt(x_head**2 + y_head**2)
distance_tail = np.sqrt(x_tail**2 + y_tail**2)

# 指定された区間で均等に分割
bin_edges = np.linspace(0, np.max([np.max(mod_head), np.max(mod_tail)]), num_bins + 1)

# ヒストグラムの作成
hist_head = np.histogram(mod_head, bins=bin_edges)
hist_tail = np.histogram(mod_tail, bins=bin_edges)

# Plotlyを使用してヒストグラムをプロット
fig = go.Figure()

# Head Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_head[1][:-1], y=hist_head[0], name="head "+entities_dict[head_id], opacity=1.0, width=0.05))

# Tail Entityのヒストグラム
fig.add_trace(go.Bar(x=hist_tail[1][:-1], y=hist_tail[0], name="tail "+entities_dict[tail_id], opacity=1.0, width=0.05))

# レイアウトの設定
fig.update_layout(
    title='HAKE',
    xaxis=dict(title='modulus 絶対値の大きさ'),
    yaxis=dict(title='要素 [hm/tm]i のカウント'),
    barmode='overlay',  # HeadとTailのヒストグラムをオーバーレイ
)

# プロットの表示
fig.show()


In [67]:
# 平均距離の計算
mean_distance_head = np.mean(np.sqrt(x_head**2 + y_head**2))
mean_distance_tail = np.mean(np.sqrt(x_tail**2 + y_tail**2))

# 結果の出力
print(f"平均距離（Head）: {mean_distance_head}")
print(f"平均距離（Tail）: {mean_distance_tail}")

平均距離（Head）: 3.9466278553009033
平均距離（Tail）: 3.886031150817871


In [68]:
# カウントの合計数を計算
head_count = np.sum(hist_head[0])
tail_count = np.sum(hist_tail[0])

# 結果の出力
print(f"表示Entity（Head）: {head_count}")
print(f"表示Entity（Tail）: {tail_count}")

表示Entity（Head）: 229
表示Entity（Tail）: 215


In [69]:

# headとtailの各座標を計算
phase_head, mod_head = np.split(entity_embedding[head_id], 2)
phase_tail, mod_tail = np.split(entity_embedding[tail_id], 2)

print("entity_embedding shape:", phase_head.shape)
print(f"phase_head: {entity_embedding[head_id],}")

entity_embedding shape: (500,)
phase_head: (array([-2.07721628e-02, -2.48875027e-03, -1.84334368e-02,  4.43110708e-03,
        1.39369280e-03, -7.74613023e-03, -5.45757590e-03,  2.68343780e-02,
        6.26703678e-03,  1.64889302e-02, -4.68467455e-03, -1.37389423e-02,
       -1.95248965e-02,  2.79028993e-03,  8.91462434e-03,  2.39546467e-02,
       -2.80327257e-02,  1.17590884e-02, -1.79714710e-02, -9.30941291e-03,
       -9.03109182e-03, -2.22332850e-02,  1.15817543e-02, -9.18311067e-03,
        7.23202433e-03, -4.37001185e-03,  1.61780547e-02,  1.22586992e-02,
        2.65974607e-02, -1.95346866e-03,  2.48360913e-02,  4.49896930e-03,
       -7.67488359e-03, -9.75729711e-03, -1.83101813e-03, -1.65270288e-02,
       -1.55140106e-02, -5.61848236e-03, -6.37261011e-03,  1.78561807e-02,
       -6.81975624e-03,  1.51347322e-02, -1.75547060e-02,  4.58768848e-03,
        9.30110738e-03,  1.56829394e-02, -1.14651117e-02, -2.36794539e-03,
        1.17598707e-02, -2.18717894e-03, -8.29946995e-03

In [70]:
# エンティティの埋め込みをロード
relation_embedding = np.load(os.path.join(model_path, 'relation_embedding.npy'))


print(f"relation_embedding: {relation_embedding[4],}")

relation_embedding: (array([ 0.00208791, -0.00047733, -0.00326881, ..., -0.12873876,
       -0.09408756, -0.14988665], dtype=float32),)
